In [1]:
from pathlib import Path
import pandas as pd

In [2]:
DATA_PATH = Path("Dades/All_dades.csv")



d_filtro= pd.read_csv(DATA_PATH)

In [3]:
d_filtro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Columns: 189 entries, codigo_participante to tt_o_f_fin
dtypes: float64(152), int64(13), object(24)
memory usage: 240.8+ KB


In [4]:
d_filtro.head(2)

,codigo_participante,recidiva,recidiva_exitus,diferencia_dias_reci_exit,causa_muerte,f_diag,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,...,bt_realPac,ini_bqt_rt,final_bqt_rt,qt,Tratamiento_sistemico_realizad,inicio_qmt,fecha_final_qmt,otros_tt,tt_o_f_ini,tt_o_f_fin
0,4,0,0,626.0,NaN,2023-11-03,NaN,NaN,21/07/2025,2025-07-21,...,NaN,NaN,NaN,1.0,2.0,14/02/2024,04/04/2024,NaN,NaN,NaN
1,7,0,0,244.0,1.0,2019-10-24,NaN,08/04/2021,24/06/2020,2020-06-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Mostrar columnas de tipo object
object_columns = d_filtro.select_dtypes(include='object').columns.tolist()
print(f"Total de columnas tipo object: {len(object_columns)}\n")
for i, col in enumerate(object_columns, 1):
    print(f"{i}. {col}")

Total de columnas tipo object: 24

1. f_diag
2. fecha_de_recidi
3. f_muerte
4. visita_control
5. Ultima_fecha
6. FN
7. f_1v
8. otra_histo
9. comentarios
10. recid_super_1
11. centro_tratPrima
12. f_tto_NA
13. fecha_qx
14. tec_Qx_avanz
15. reintervencion_motivo
16. otras
17. otras_especifi
18. histo_otros
19. ap_comentarios
20. ini_bqt_rt
21. final_bqt_rt
22. inicio_qmt
23. fecha_final_qmt
24. otros_tt


## tractament FETXA: 
tot igual al mateix format


In [12]:
col_fetxa= ["f_diag", "fecha_de_recidi", "f_muerte", "visita_control", "Ultima_fecha","FN", "f_1v",  "f_tto_NA",  "fecha_qx","ini_bqt_rt",  "final_bqt_rt","inicio_qmt", "fecha_final_qmt"        ]

In [13]:
df_fechas= d_filtro[col_fetxa].copy()

In [15]:
# Convertir todas las columnas de fecha a tipo datetime
for col in col_fetxa:
    if col in df_fechas.columns:
        df_fechas[col] = pd.to_datetime(df_fechas[col], errors='coerce')

print("Columnas convertidas a datetime:")
print(df_fechas.dtypes)

Columnas convertidas a datetime:
f_diag             datetime64[ns]
fecha_de_recidi    datetime64[ns]
f_muerte           datetime64[ns]
visita_control     datetime64[ns]
Ultima_fecha       datetime64[ns]
FN                 datetime64[ns]
f_1v               datetime64[ns]
f_tto_NA           datetime64[ns]
fecha_qx           datetime64[ns]
ini_bqt_rt         datetime64[ns]
final_bqt_rt       datetime64[ns]
inicio_qmt         datetime64[ns]
fecha_final_qmt    datetime64[ns]
dtype: object


C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\2166011672.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_fechas[col] = pd.to_datetime(df_fechas[col], errors='coerce')
C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\2166011672.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_fechas[col] = pd.to_datetime(df_fechas[col], errors='coerce')
C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\2166011672.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_fechas[col] = pd.to_datetime(df_fechas[col], errors='coerce')
C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\2166011672.py:4: UserWarning: Parsing dates in %d/%m/%Y format wh

In [16]:
df_fechas.head(3)

,f_diag,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,FN,f_1v,f_tto_NA,fecha_qx,ini_bqt_rt,final_bqt_rt,inicio_qmt,fecha_final_qmt
0,2023-11-03,NaT,NaT,2025-07-21,2025-07-21,1955-12-10,2023-03-11,NaT,2023-11-16,NaT,NaT,2024-02-14,2024-04-04
1,2019-10-24,NaT,2021-08-04,2020-06-24,2020-06-24,NaT,2019-11-04,NaT,NaT,NaT,NaT,NaT,NaT
2,2020-07-16,NaT,NaT,2025-05-20,2025-05-20,NaT,NaT,NaT,2020-09-14,NaT,NaT,NaT,NaT


In [17]:
# Calcular intervalos de tratamiento en días y un intervalo combinado
# Requiere que las columnas de fecha estén ya en datetime64[ns]
from datetime import datetime

def calcular_intervalo_en_dias(df, inicio_col, fin_col, nuevo_col, estado_col=None):
    """
    Calcula días entre inicio y fin.
    - Si falta fecha de fin pero hay inicio: usa hoy y marca estado "en_tratamiento".
    - Si faltan ambas: deja NaN.
    - Si fin < inicio: deja NaN.
    Opcional: crea columna de estado con valores {"completado", "en_tratamiento", "sin_datos= -1"}.
    """
    hoy = pd.Timestamp(datetime.today().date())

    if inicio_col in df.columns and fin_col in df.columns:
        inicio = df[inicio_col]
        fin = df[fin_col]

        # Estado por fila
        if estado_col:
            estado = pd.Series(index=df.index, dtype='object')
            estado[(inicio.notna()) & (fin.notna())] = 'completado'
            estado[(inicio.notna()) & (fin.isna())] = 'en_tratamiento'
            estado[(inicio.isna())] = 'NaT'
            df[estado_col] = estado

        # Calcular días: usar fin o hoy si fin está NaT y inicio existe
        fin_efectivo = fin.copy()
        fin_efectivo[(fin.isna()) & (inicio.notna())] = hoy

        dias = (fin_efectivo - inicio).dt.days
        # invalidar casos donde fin < inicio
        dias[(fin_efectivo.notna()) & (inicio.notna()) & (fin_efectivo < inicio)] = pd.NA
        df[nuevo_col] = dias
    else:
        print(f"Columnas {inicio_col} o {fin_col} no encontradas en el DataFrame.")

# BQT/RT
calcular_intervalo_en_dias(df_fechas, 'ini_bqt_rt', 'final_bqt_rt', 'intervalo_bqt_rt_dias', estado_col='estado_bqt_rt')
# QMT
calcular_intervalo_en_dias(df_fechas, 'inicio_qmt', 'fecha_final_qmt', 'intervalo_qmt_dias', estado_col='estado_qmt')

# Vista rápida
cols_show = [c for c in ['ini_bqt_rt','final_bqt_rt','intervalo_bqt_rt_dias','estado_bqt_rt','inicio_qmt','fecha_final_qmt','intervalo_qmt_dias','estado_qmt'] if c in df_fechas.columns]
df_fechas[cols_show].head(10)


C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\1016233463.py:33: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  dias[(fin_efectivo.notna()) & (inicio.notna()) & (fin_efectivo < inicio)] = pd.NA
C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\1016233463.py:33: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  dias[(fin_efectivo.notna()) & (inicio.notna()) & (fin_efectivo < inicio)] = pd.NA
C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\1016233463.py:33: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  dias[(fin_efectivo.notna()) & (inicio.notna()) & (fin_efectivo < inicio)] = pd.NA
C:\Users\sylbo\AppData\Local\Temp\ipykernel_30168\1016233463.py:33: SettingWithCopyWarning: modifications to 

,ini_bqt_rt,final_bqt_rt,intervalo_bqt_rt_dias,estado_bqt_rt,inicio_qmt,fecha_final_qmt,intervalo_qmt_dias,estado_qmt
0,NaT,NaT,NaN,NaT,2024-02-14,2024-04-04,50.0,completado
1,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT
2,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT
3,2021-08-10,2021-12-11,123.0,completado,2021-05-06,NaT,1682.0,en_tratamiento
4,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT
5,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT
6,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT
7,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT
8,2022-02-03,2024-07-03,881.0,completado,NaT,NaT,NaN,NaT
9,NaT,NaT,NaN,NaT,NaT,NaT,NaN,NaT


In [18]:
df_fechas['estado_bqt_rt'].value_counts()

estado_bqt_rt
NaT               144
completado         13
en_tratamiento      6
Name: count, dtype: int64

In [19]:
df_fechas['estado_qmt'].value_counts()

estado_qmt
NaT               139
en_tratamiento     14
completado         10
Name: count, dtype: int64

**NEW COL:**
- **ESTADO TRATAMIENTO**:estado_qmt i estado_bqt_rt  
- **Durada tratamiento**: intervalo_qmt_dias, intervalo_bqt_rt_dias

totes dades Nan --> passar a '-1'  
[per a mateixa expresió que numeriques]

In [20]:
# Las columnas de fecha mantienen NaT para valores faltantes (tipo datetime)
# Las columnas de estado se pueden convertir después si es necesario

print("Valores únicos en estado_bqt_rt:", df_fechas['estado_bqt_rt'].unique())
print("Valores únicos en estado_qmt:", df_fechas['estado_qmt'].unique())
print("\nPrimeras filas:")
df_fechas.head()

Valores únicos en estado_bqt_rt: ['NaT' 'completado' 'en_tratamiento']
Valores únicos en estado_qmt: ['completado' 'NaT' 'en_tratamiento']

Primeras filas:


,f_diag,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,FN,f_1v,f_tto_NA,fecha_qx,ini_bqt_rt,final_bqt_rt,inicio_qmt,fecha_final_qmt,estado_bqt_rt,intervalo_bqt_rt_dias,estado_qmt,intervalo_qmt_dias
0,2023-11-03,NaT,NaT,2025-07-21,2025-07-21,1955-12-10,2023-03-11,NaT,2023-11-16,NaT,NaT,2024-02-14,2024-04-04,NaT,NaN,completado,50.0
1,2019-10-24,NaT,2021-08-04,2020-06-24,2020-06-24,NaT,2019-11-04,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaN
2,2020-07-16,NaT,NaT,2025-05-20,2025-05-20,NaT,NaT,NaT,2020-09-14,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaN
3,2021-02-22,2023-11-22,NaT,2025-10-27,2023-11-22,NaT,2021-11-02,NaT,2021-03-18,2021-08-10,2021-12-11,2021-05-06,NaT,completado,123.0,en_tratamiento,1682.0
4,2019-11-15,NaT,NaT,2025-02-13,2025-02-13,1951-06-07,2019-12-11,NaT,2019-12-13,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaN


In [21]:
df_fechas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   f_diag                 161 non-null    datetime64[ns]
 1   fecha_de_recidi        29 non-null     datetime64[ns]
 2   f_muerte               14 non-null     datetime64[ns]
 3   visita_control         163 non-null    datetime64[ns]
 4   Ultima_fecha           163 non-null    datetime64[ns]
 5   FN                     69 non-null     datetime64[ns]
 6   f_1v                   66 non-null     datetime64[ns]
 7   f_tto_NA               3 non-null      datetime64[ns]
 8   fecha_qx               150 non-null    datetime64[ns]
 9   ini_bqt_rt             19 non-null     datetime64[ns]
 10  final_bqt_rt           22 non-null     datetime64[ns]
 11  inicio_qmt             24 non-null     datetime64[ns]
 12  fecha_final_qmt        10 non-null     datetime64[ns]
 13  estad

In [22]:
# Quitar horas de todas las columnas datetime (dejar solo fechas)
# Normalizar a medianoche (00:00:00)

datetime_cols_df_fechas = df_fechas.select_dtypes(include=['datetime64']).columns

print(f"Columnas datetime en df_fechas: {list(datetime_cols_df_fechas)}")

# Normalizar en df_fechas
for col in datetime_cols_df_fechas:
    df_fechas[col] = df_fechas[col].dt.normalize()

print("\nFechas normalizadas (sin hora). Vista:")
df_fechas[datetime_cols_df_fechas].head() if len(datetime_cols_df_fechas) > 0 else print("No hay columnas datetime en df_fechas")

Columnas datetime en df_fechas: ['f_diag', 'fecha_de_recidi', 'f_muerte', 'visita_control', 'Ultima_fecha', 'FN', 'f_1v', 'f_tto_NA', 'fecha_qx', 'ini_bqt_rt', 'final_bqt_rt', 'inicio_qmt', 'fecha_final_qmt']

Fechas normalizadas (sin hora). Vista:


,f_diag,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,FN,f_1v,f_tto_NA,fecha_qx,ini_bqt_rt,final_bqt_rt,inicio_qmt,fecha_final_qmt
0,2023-11-03,NaT,NaT,2025-07-21,2025-07-21,1955-12-10,2023-03-11,NaT,2023-11-16,NaT,NaT,2024-02-14,2024-04-04
1,2019-10-24,NaT,2021-08-04,2020-06-24,2020-06-24,NaT,2019-11-04,NaT,NaT,NaT,NaT,NaT,NaT
2,2020-07-16,NaT,NaT,2025-05-20,2025-05-20,NaT,NaT,NaT,2020-09-14,NaT,NaT,NaT,NaT
3,2021-02-22,2023-11-22,NaT,2025-10-27,2023-11-22,NaT,2021-11-02,NaT,2021-03-18,2021-08-10,2021-12-11,2021-05-06,NaT
4,2019-11-15,NaT,NaT,2025-02-13,2025-02-13,1951-06-07,2019-12-11,NaT,2019-12-13,NaT,NaT,NaT,NaT


## Mirar resta de columnes OBJ: 

In [23]:
rest_col_obj= d_filtro.select_dtypes(include='object').columns.difference(df_fechas.columns).tolist()
rest_col_obj
##TOTES SON DESCRIPTIVES DE TEXT --> GUARDAR EN csv separat i despres tractar 

['ap_comentarios',
 'centro_tratPrima',
 'comentarios',
 'histo_otros',
 'otra_histo',
 'otras',
 'otras_especifi',
 'otros_tt',
 'recid_super_1',
 'reintervencion_motivo',
 'tec_Qx_avanz']

In [24]:
# Construir df_descriptivas_text con rest_col_obj + codigo_participante
# Asegurar existencia de lista rest_col_obj y columna codigo_participante
if 'rest_col_obj' not in globals():
    print("Advertencia: 'rest_col_obj' no está definido. Define la lista de columnas de texto.")
else:
    cols_norm = [c.strip() for c in rest_col_obj]
    base_cols = [c for c in cols_norm if c in d_filtro.columns]
    # añadir codigo_participante si existe
    id_col = 'codigo_participante'
    if id_col in d_filtro.columns:
        final_cols = [id_col] + base_cols
    else:
        final_cols = base_cols
        print("Advertencia: 'codigo_participante' no se encontró en d_filtro.")

    df_descriptivas_text = d_filtro[final_cols].copy()
    print(f"Columnas incluidas: {final_cols}")
    # Vista rápida
    df_descriptivas_text.head()

Columnas incluidas: ['codigo_participante', 'ap_comentarios', 'centro_tratPrima', 'comentarios', 'histo_otros', 'otra_histo', 'otras', 'otras_especifi', 'otros_tt', 'recid_super_1', 'reintervencion_motivo', 'tec_Qx_avanz']


In [25]:
df_descriptivas_text.head()

,codigo_participante,ap_comentarios,centro_tratPrima,comentarios,histo_otros,otra_histo,otras,otras_especifi,otros_tt,recid_super_1,reintervencion_motivo,tec_Qx_avanz
0,4,"OVARIO Y TROMPA DE FALLOPIO IZQUIERDOS, ANEXEC...",NaN,estad�o IV por diseminaci�n pulmonar (varios ...,dudosa entre carcinoma seroso o endometrioide.,NaN,NaN,Paciente ingresa a urgencias por oclusi�n inte...,NaN,NaN,Reintervenci�n por parte de cirugia general po...,NaN
1,7,NaN,No iQx como tto primario. comorbilidades varias.,Es notifica que la pacient rebutja la realitza...,NaN,y escasos focos de endocarcinoma endometroide,NaN,NaN,NaN,NaN,NaN,NaN
2,8,"ADENOPAT�A MESENTERIO, RESECCI�N: - UN GANGLIO...",NaN,en paralel dx 2020 linfoma linfocitic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,"VAGINA, AMPLIACI�N MARGEN LATERAL IZQUIERDO: -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,recidiva adenopatica. ttm SBRT,NaN,NaN
4,10,NaN,NaN,Alta hospital�ria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# GUARDAR ELS DATASETS 
- DATASET: var descriptives de text
- Dataset: tractat dates (amb noves variables)


In [26]:
df_fechas.columns

Index(['f_diag', 'fecha_de_recidi', 'f_muerte', 'visita_control',
       'Ultima_fecha', 'FN', 'f_1v', 'f_tto_NA', 'fecha_qx', 'ini_bqt_rt',
       'final_bqt_rt', 'inicio_qmt', 'fecha_final_qmt', 'estado_bqt_rt',
       'intervalo_bqt_rt_dias', 'estado_qmt', 'intervalo_qmt_dias'],
      dtype='object')

In [27]:
#save df_text --> csv individual

OUT_CSV = Path("Dades/dades_textuals_descriptives.csv")
df_descriptivas_text.to_csv(OUT_CSV, index=False)


In [28]:
# Guardar dataset completo sin columnas descriptivas de texto
# Incluye fechas procesadas, intervalos, estados, y todas las numéricas

# Columnas a excluir (las descriptivas de texto)
cols_excluir = set(rest_col_obj) if 'rest_col_obj' in globals() else set()

# Columnas a mantener
cols_mantener = [c for c in d_filtro.columns if c not in cols_excluir]

# Crear dataframe final
df_completo = d_filtro[cols_mantener].copy()

# Añadir las nuevas columnas de intervalos y estados desde df_fechas si no están ya
if 'df_fechas' in globals():
    nuevas_cols = ['intervalo_bqt_rt_dias','estado_bqt_rt','intervalo_qmt_dias','estado_qmt']
    for nc in nuevas_cols:
        if nc in df_fechas.columns and nc not in df_completo.columns:
            df_completo[nc] = df_fechas[nc]


print(f"Total de columnas en dataset completo: {len(df_completo.columns)}")
print(f"Columnas excluidas (texto): {len(cols_excluir)}")

# Guardar CSV
OUT_CSV_COMPLETO = Path("Dades/datos_preprocesados.csv")
df_completo.to_csv(OUT_CSV_COMPLETO, index=False)
print(f"\nDataset guardado en: {OUT_CSV_COMPLETO}")

df_completo.head()

Total de columnas en dataset completo: 182
Columnas excluidas (texto): 11

Dataset guardado en: Dades\datos_preprocesados.csv


,codigo_participante,recidiva,recidiva_exitus,diferencia_dias_reci_exit,causa_muerte,f_diag,fecha_de_recidi,f_muerte,visita_control,Ultima_fecha,...,qt,Tratamiento_sistemico_realizad,inicio_qmt,fecha_final_qmt,tt_o_f_ini,tt_o_f_fin,intervalo_bqt_rt_dias,estado_bqt_rt,intervalo_qmt_dias,estado_qmt
0,4,0,0,626.0,NaN,2023-11-03,NaN,NaN,21/07/2025,2025-07-21,...,1.0,2.0,14/02/2024,04/04/2024,NaN,NaN,NaN,NaT,50.0,completado
1,7,0,0,244.0,1.0,2019-10-24,NaN,08/04/2021,24/06/2020,2020-06-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT
2,8,0,0,1769.0,NaN,2020-07-16,NaN,NaN,20/05/2025,2025-05-20,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT
3,9,1,1,1003.0,NaN,2021-02-22,22/11/2023,NaN,27/10/2025,2023-11-22,...,1.0,2.0,06/05/2021,20/08/2021,NaN,NaN,123.0,completado,1682.0,en_tratamiento
4,10,0,0,1917.0,NaN,2019-11-15,NaN,NaN,13/02/2025,2025-02-13,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT


In [29]:
df_completo.columns

Index(['codigo_participante', 'recidiva', 'recidiva_exitus',
       'diferencia_dias_reci_exit', 'causa_muerte', 'f_diag',
       'fecha_de_recidi', 'f_muerte', 'visita_control', 'Ultima_fecha',
       ...
       'qt', 'Tratamiento_sistemico_realizad', 'inicio_qmt', 'fecha_final_qmt',
       'tt_o_f_ini', 'tt_o_f_fin', 'intervalo_bqt_rt_dias', 'estado_bqt_rt',
       'intervalo_qmt_dias', 'estado_qmt'],
      dtype='object', length=182)